# Stock Trading Bot Ayyyyy
- api reference: https://www.alphavantage.co/documentation/ and potentially https://site.financialmodelingprep.com/developer/docs

In [1]:
!pip install graphviz python-decouple

In [5]:
# This is just importing certain libraries we will use consistently throughouit the porject
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import requests

In [3]:
import config

api_key = config.api_key
base_url = "https://www.alphavantage.co/query?"

In [6]:
ticker_function = "TIME_SERIES_DAILY"
symbol = "WBD"
ticker_url = f"{base_url}function={ticker_function}&symbol={symbol}&apikey={api_key}"
ticker_response = requests.get(ticker_url)
ticker_data = ticker_response.json()
data = pd.DataFrame.from_dict(ticker_data["Time Series (Daily)"], orient="index")
data.head()

,1. open,2. high,3. low,4. close,5. volume
2023-09-29,10.7700,11.0700,10.7700,10.8600,19106925
2023-09-28,11.0800,11.1000,10.6600,10.6900,17806298
2023-09-27,10.7600,11.1150,10.7600,11.0300,20955284
2023-09-26,10.5700,10.8100,10.4250,10.6900,21457864
2023-09-25,11.1500,11.1800,10.6200,10.6600,22405085


In [153]:
# # Feature engineering (you can add more relevant features)
data['4. close'] = data['4. close'].astype(float)
data['1. open'] = data['1. open'].astype(float)

In [154]:
print(data.dtypes)

1. open      float64
2. high       object
3. low        object
4. close     float64
5. volume     object
dtype: object


In [158]:
data['Close_Prev_Day'] = data['4. close'].shift(1)

# Then, create 'Open_Next_Day' by shifting 'Open' one day forward
data['Open_Next_Day'] = data['1. open'].shift(-1)

# Calculate the difference between 'Close_Prev_Day' and 'Open_Next_Day'
data['Price_Diff'] = data['Open_Next_Day'] - data['Close_Prev_Day']

# Drop the NaN values in the new column, which will be present in the first and last rows
data.dropna(subset=['Price_Diff'], inplace=True)
data = data.head(50)

In [159]:
maximum = max(data['Price_Diff'])
data[data['Price_Diff'] == maximum]

,1. open,2. high,3. low,4. close,5. volume,Close_Prev_Day,Open_Next_Day,Price_Diff
2023-09-01,12.78,12.8400,11.3400,11.56,62845901,11.64,13.1,1.46


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Labeling data (creating a simple binary label: 1 for buy, 0 for not buy)
maximum = max(data[0], data[-1])

# Prepare features and labels
X = data[['SMA_50', 'SMA_200', 'RSI']]
y = data['Buy']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [43]:
# for ticker in tickers:
#     historical_endpoint = "historical-price-full"
#     historical_url = f"{base_url}{historical_endpoint}/{ticker}?apikey={api_key}"
#     historical_response = requests.get(historical_url)
#     historical_data = historical_response.json()

In [18]:
# import concurrent.futures

# def fetch_historical_data(ticker):
#     """Fetches the data from the API and returns it as a JSON response."""
#     historical_url = f"{base_url}{historical_endpoint}/{ticker}?apikey={api_key}"
#     response = requests.get(historical_url)
#     data = response.json()
#     return data

# # This is modified based on the hardware of the computer. 4-8 threads for a cpu is common so I defaulted to 8.
# max_workers = 8

# # Create a ThreadPoolExecutor with max_workers threads
# with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#     # Submit the fetch_historical_data function for each ticker
#     futures = [executor.submit(fetch_historical_data, ticker) for ticker in tickers]

#     # Process the results as they complete
#     for future in concurrent.futures.as_completed(futures):
#         try:
#             data = future.result()
#             # Process the fetched historical data as needed
#             print(data)
#         except Exception as e:
#             print(f"An error occurred: {e}")
